In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import plotly.express as px
import plotly.graph_objects as go
import os
from docx import Document
from tqdm import tqdm
from collections import Counter
import seaborn as sns
import utils
import matplotlib.pyplot as plt

# reload utils
import importlib
importlib.reload(utils)

<module 'utils' from '/Users/northstar/Projects/portfolio/utils.py'>

In [2]:
curr_path = os.path.dirname(os.path.realpath(__name__))
data_dir = os.path.join(curr_path, 'data')

# list dir without .DS_Store
data_folders = [f for f in os.listdir(data_dir) if not f.startswith('.')]

In [3]:
data_folders

['benchmark',
 'arbitrage',
 'multi_asset',
 'conservative_hybrid',
 'equity_savings',
 'aggressive_hybrid']

In [4]:
dfs = {}
for data_folder in data_folders:
    dfs[data_folder] = {}
    curr_path = os.path.join(data_dir, data_folder)
    files = [f for f in os.listdir(curr_path) if not f.startswith('.')]
    print(f"Processing {data_folder} folder")
    for file in tqdm(files):
        if file.endswith('.docx'):
            doc = Document(os.path.join(curr_path, file))
            table = doc.tables[0]
            data = []
            for row in table.rows:
                data.append([cell.text for cell in row.cells])

            df = pd.DataFrame(data)

            # rename columns and drop first 5 rows
            df.columns = df.iloc[4].values
            df = df[5:]
            df.index = df['NAV date']
            df.index.name = 'date'
            indexes = []
            for ind in df.index:
                indexes.append(ind.split('<')[0])
            df.index = indexes
            # display(df.head(10))
            # print(df.index[:10])
            try:
                df.index = pd.to_datetime(df.index)
                
            except:
                try:
                    df.index = pd.to_datetime(df.index, format="%d-%b-%Y")
                except:
                    print(f"type of index: {type(df.index[0])}")
                    display(df.head(1))
                    print(f"Error converting date in {file}")
    
            df = df.iloc[:, :1]
            df.columns = ["NAV"]
            # set NAV columns as float type
            df['NAV'] = df['NAV'].str.replace(',', '').astype(float)
            # df = df.add_suffix(f'_{file.split(".")[0]}')
            df = df[~df.index.duplicated(keep='last')]
            
            dfs[data_folder][file.split('.')[0]] = df

Processing benchmark folder


100%|██████████| 1/1 [00:00<00:00, 29537.35it/s]


Processing arbitrage folder


100%|██████████| 5/5 [00:01<00:00,  4.43it/s]


Processing multi_asset folder


...

100%|██████████| 5/5 [00:01<00:00,  4.19it/s]


Processing equity_savings folder


100%|██████████| 5/5 [00:01<00:00,  4.21it/s]


Processing aggressive_hybrid folder


100%|██████████| 5/5 [00:01<00:00,  4.07it/s]


In [5]:
dfs.keys()

dict_keys(['benchmark', 'arbitrage', 'multi_asset', 'conservative_hybrid', 'equity_savings', 'aggressive_hybrid'])

In [6]:
dfs['arbitrage'].keys()

dict_keys(['Invesco_India_Arbitrage_Fund', 'Kotak_Equity_Arbitrage_Fund', 'SBI_Arbitrage_Opportunities_Fund', 'ICICI_Prudential_Equity_Arbitrage_Fund', 'HDFC_Arbitrage_Fund_Wholesale_Plan'])

In [7]:
dfs['arbitrage']['Invesco_India_Arbitrage_Fund']

,NAV
2020-03-02,24.8453
2020-03-03,24.8360
2020-03-04,24.8369
2020-03-05,24.8587
2020-03-06,24.8667
...,...
2025-02-14,33.5898
2025-02-17,33.5797
2025-02-18,33.6183
2025-02-19,33.6177


In [8]:
for df in dfs['arbitrage'].values():

    print(len(df.index))

1229
1229
1232
1229
1229


In [9]:
for big_key in dfs.keys():
    print(f"{big_key}:")
    for key, df in dfs[big_key].items():
        print(f"{key}: {len(df.index)}")

benchmark:
arbitrage:
Invesco_India_Arbitrage_Fund: 1229
Kotak_Equity_Arbitrage_Fund: 1229
SBI_Arbitrage_Opportunities_Fund: 1232
ICICI_Prudential_Equity_Arbitrage_Fund: 1229
HDFC_Arbitrage_Fund_Wholesale_Plan: 1229
multi_asset:
SBI_Multi_Asset_Allocation_Fund: 1209
ICICI_Prudential_Multi_Asset_Fund: 1229
HDFC_Multi_Asset_Fund: 1229
Nippon_India_Multi_Asset_Allocation_Fund: 1106
UTI_Multi_Asset_Allocation_Fund: 1229
conservative_hybrid:
HDFC_Hybrid_Debt_Fund: 1207
SBI_Conservative_Hybrid_Fund: 1209
Kotak_Debt_Hybrid_Fund: 1206
ICICI_Prudential_Regular_Savings_Fund: 1206
UTI_Conservative_Hybrid_Fund: 1206
equity_savings:
ICICI_Prudential_Equity_Savings_Fund: 1229
HDFC_Equity_Savings_Fund: 1229
Kotak_Equity_Savings_Fund: 1229
SBI_Equity_Savings_Fund: 1232
DSP_Equity_Savings_Fund: 1229
aggressive_hybrid:
Canara_Robeco_Equity_Hybrid_fund: 1229
DSP_Aggresive_Hybrid_Fund: 1229
HDFC_Hybrid_Equity_Fund: 1229
SBI_Equity_Hybrid_Fund: 1232
ICICI_Prudential_Equity_and_Debt_Fund: 1229


In [10]:
nifty_df = pd.read_csv('data/benchmark/nifty_50.csv', index_col='Date')
nifty_df.index = pd.to_datetime(nifty_df.index)
nifty_df.sort_index(inplace=True)
nifty_df['NAV_benchmark_nifty'] = nifty_df['Close']
nifty_df = nifty_df[['NAV_benchmark_nifty']]
nifty_df.index.name = 'date'
nifty_df

,NAV_benchmark_nifty
date,
2020-08-31,11387.50
2020-09-01,11470.25
2020-09-02,11535.00
2020-09-03,11527.45
2020-09-04,11333.85
...,...
2025-02-14,22929.25
2025-02-17,22959.50
2025-02-18,22945.30


In [11]:
dfs['benchmark'] = {}
dfs['benchmark']['nifty_50'] = nifty_df

In [12]:
merged_df = None
for big_key in dfs.keys():
    # print(f"Processing {big_key}")
    for key, df in tqdm(dfs[big_key].items()):
        df = df.copy().add_suffix(f'_{big_key}_{key}')
        if merged_df is None:
            merged_df = df
        else:
            merged_df = merged_df.merge(df, how='outer', left_index=True, right_index=True)

merged_df.dropna(inplace=True)
merged_df.sort_index(inplace=True)
merged_df.index.name = 'date'

# divide each column with their first value
# norm_df = merged_df.div(merged_df.iloc[0]) * 100
# norm_df = merged_df.div(merged_df.loc['2024-08-30']) * 100
norm_df = merged_df.div(merged_df.loc['2022-01-03']) * 100
# norm_df = merged_df.div(merged_df.loc['2021-01-01']) * 100
# norm_df = merged_df.div(merged_df.loc['2025-01-01']) * 100

weights = pd.Series()

# Uniform Weights
for column in norm_df.columns:
    if "benchmark" in column:
        weights[column] = 0
    else:
        weights[column] = 1 / (len(norm_df.columns) - 1)

# Uniform weights to top funds in each category
for column in norm_df.columns:
    weights[column] = 0
    
weights["NAV_arbitrage_Invesco_India_Arbitrage_Fund"] = 0.1
weights["NAV_multi_asset_ICICI_Prudential_Multi_Asset_Fund"] = 0.4
weights["NAV_conservative_hybrid_ICICI_Prudential_Regular_Savings_Fund"] = 0.1
weights["NAV_equity_savings_ICICI_Prudential_Equity_Savings_Fund"] = 0.1
weights["NAV_aggressive_hybrid_ICICI_Prudential_Equity_and_Debt_Fund"] = 0.3

portfolio = pd.Series(np.zeros(len(norm_df)),index=norm_df.index)
for column in norm_df.columns:
    portfolio += norm_df[column] * weights[column]

norm_df['portfolio'] = portfolio

print(f"Number of weights: {len(weights)}")
display(weights[weights != 0])

100%|██████████| 5/5 [00:00<00:00, 1155.77it/s]

Number of weights: 26


NAV_arbitrage_Invesco_India_Arbitrage_Fund                       0.1
NAV_multi_asset_ICICI_Prudential_Multi_Asset_Fund                0.4
NAV_conservative_hybrid_ICICI_Prudential_Regular_Savings_Fund    0.1
NAV_equity_savings_ICICI_Prudential_Equity_Savings_Fund          0.1
NAV_aggressive_hybrid_ICICI_Prudential_Equity_and_Debt_Fund      0.3
dtype: float64

In [13]:
merged_df

,NAV_benchmark_nifty_benchmark_nifty_50,NAV_arbitrage_Invesco_India_Arbitrage_Fund,NAV_arbitrage_Kotak_Equity_Arbitrage_Fund,NAV_arbitrage_SBI_Arbitrage_Opportunities_Fund,NAV_arbitrage_ICICI_Prudential_Equity_Arbitrage_Fund,NAV_arbitrage_HDFC_Arbitrage_Fund_Wholesale_Plan,NAV_multi_asset_SBI_Multi_Asset_Allocation_Fund,NAV_multi_asset_ICICI_Prudential_Multi_Asset_Fund,NAV_multi_asset_HDFC_Multi_Asset_Fund,NAV_multi_asset_Nippon_India_Multi_Asset_Allocation_Fund,...,NAV_equity_savings_ICICI_Prudential_Equity_Savings_Fund,NAV_equity_savings_HDFC_Equity_Savings_Fund,NAV_equity_savings_Kotak_Equity_Savings_Fund,NAV_equity_savings_SBI_Equity_Savings_Fund,NAV_equity_savings_DSP_Equity_Savings_Fund,NAV_aggressive_hybrid_Canara_Robeco_Equity_Hybrid_fund,NAV_aggressive_hybrid_DSP_Aggresive_Hybrid_Fund,NAV_aggressive_hybrid_HDFC_Hybrid_Equity_Fund,NAV_aggressive_hybrid_SBI_Equity_Hybrid_Fund,NAV_aggressive_hybrid_ICICI_Prudential_Equity_and_Debt_Fund
date,,,,,,,,,,,,,,,,,,,,,
2020-08-31,11387.50,25.4211,29.5394,26.7231,27.3938,15.078,32.1628,287.8636,37.309,9.9220,...,15.15,39.897,15.9160,14.7307,13.740,184.65,173.529,55.886,151.2363,142.45
2020-09-01,11470.25,25.3963,29.5134,26.6938,27.3812,15.058,32.4572,291.2120,37.488,9.9562,...,15.20,40.036,15.9548,14.7996,13.807,185.71,174.930,56.125,152.6903,143.93
2020-09-02,11535.00,25.4084,29.5244,26.7084,27.3935,15.067,32.5219,291.9228,37.622,10.0002,...,15.24,40.126,16.0017,14.8511,13.851,186.78,175.594,56.378,153.6755,144.42
2020-09-03,11527.45,25.4188,29.5370,26.7201,27.4074,15.072,32.6015,289.9180,37.565,9.9214,...,15.23,40.094,16.0169,14.8543,13.852,186.84,175.436,56.298,153.4914,143.71
2020-09-04,11333.85,25.4265,29.5504,26.7204,27.4153,15.078,32.3780,285.4421,37.205,9.8052,...,15.14,39.768,15.9315,14.7546,13.782,185.16,173.647,55.494,151.5306,141.64
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-02-13,23031.40,33.5721,38.9609,34.9659,35.7932,19.636,60.4248,772.1037,73.649,20.9540,...,23.17,69.808,26.9512,24.6317,23.309,370.30,371.043,118.515,299.8113,395.68
2025-02-14,22929.25,33.5898,38.9752,34.9810,35.8078,19.642,60.2079,769.5966,73.397,20.8358,...,23.14,69.594,26.7976,24.4879,23.249,366.50,367.602,118.026,298.3150,393.00
2025-02-17,22959.50,33.5797,38.9679,34.9729,35.8005,19.640,60.0667,770.4111,73.382,20.7984,...,23.15,69.669,26.8137,24.4576,23.232,366.69,367.209,117.851,297.9564,393.39


In [14]:
merged_df.columns

Index(['NAV_benchmark_nifty_benchmark_nifty_50',
       'NAV_arbitrage_Invesco_India_Arbitrage_Fund',
       'NAV_arbitrage_Kotak_Equity_Arbitrage_Fund',
       'NAV_arbitrage_SBI_Arbitrage_Opportunities_Fund',
       'NAV_arbitrage_ICICI_Prudential_Equity_Arbitrage_Fund',
       'NAV_arbitrage_HDFC_Arbitrage_Fund_Wholesale_Plan',
       'NAV_multi_asset_SBI_Multi_Asset_Allocation_Fund',
       'NAV_multi_asset_ICICI_Prudential_Multi_Asset_Fund',
       'NAV_multi_asset_HDFC_Multi_Asset_Fund',
       'NAV_multi_asset_Nippon_India_Multi_Asset_Allocation_Fund',
       'NAV_multi_asset_UTI_Multi_Asset_Allocation_Fund',
       'NAV_conservative_hybrid_HDFC_Hybrid_Debt_Fund',
       'NAV_conservative_hybrid_SBI_Conservative_Hybrid_Fund',
       'NAV_conservative_hybrid_Kotak_Debt_Hybrid_Fund',
       'NAV_conservative_hybrid_ICICI_Prudential_Regular_Savings_Fund',
       'NAV_conservative_hybrid_UTI_Conservative_Hybrid_Fund',
       'NAV_equity_savings_ICICI_Prudential_Equity_Savings_Fund',

In [15]:
norm_df

,NAV_benchmark_nifty_benchmark_nifty_50,NAV_arbitrage_Invesco_India_Arbitrage_Fund,NAV_arbitrage_Kotak_Equity_Arbitrage_Fund,NAV_arbitrage_SBI_Arbitrage_Opportunities_Fund,NAV_arbitrage_ICICI_Prudential_Equity_Arbitrage_Fund,NAV_arbitrage_HDFC_Arbitrage_Fund_Wholesale_Plan,NAV_multi_asset_SBI_Multi_Asset_Allocation_Fund,NAV_multi_asset_ICICI_Prudential_Multi_Asset_Fund,NAV_multi_asset_HDFC_Multi_Asset_Fund,NAV_multi_asset_Nippon_India_Multi_Asset_Allocation_Fund,...,NAV_equity_savings_HDFC_Equity_Savings_Fund,NAV_equity_savings_Kotak_Equity_Savings_Fund,NAV_equity_savings_SBI_Equity_Savings_Fund,NAV_equity_savings_DSP_Equity_Savings_Fund,NAV_aggressive_hybrid_Canara_Robeco_Equity_Hybrid_fund,NAV_aggressive_hybrid_DSP_Aggresive_Hybrid_Fund,NAV_aggressive_hybrid_HDFC_Hybrid_Equity_Fund,NAV_aggressive_hybrid_SBI_Equity_Hybrid_Fund,NAV_aggressive_hybrid_ICICI_Prudential_Equity_and_Debt_Fund,portfolio
date,,,,,,,,,,,,,,,,,,,,,
2020-08-31,64.607363,94.719487,94.256431,94.603081,94.386846,94.627840,80.276150,64.784374,72.571484,73.282961,...,76.694027,81.858111,78.192164,78.834127,68.179301,66.721137,66.225056,68.527040,59.983999,70.295881
2020-09-01,65.076848,94.627081,94.173469,94.499356,94.343432,94.502322,81.010952,65.537939,72.919665,73.535559,...,76.961227,82.057665,78.557893,79.218544,68.570690,67.259815,66.508271,69.185866,60.607209,70.857327
2020-09-02,65.444209,94.672166,94.208568,94.551042,94.385813,94.558805,81.172439,65.697906,73.180315,73.860539,...,77.134234,82.298878,78.831261,79.470997,68.965772,67.515120,66.808077,69.632272,60.813542,71.023896
2020-09-03,65.401374,94.710917,94.248773,94.592461,94.433706,94.590185,81.371115,65.246721,73.069442,73.278530,...,77.072721,82.377053,78.848246,79.476734,68.987926,67.454370,66.713277,69.548854,60.514570,70.759858
2020-09-04,64.302978,94.739607,94.291531,94.593523,94.460926,94.627840,80.813274,64.239410,72.369189,72.420288,...,76.446051,81.937830,78.319028,79.075105,68.367611,66.766507,65.760535,68.660391,59.642917,70.022816
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-02-13,130.669420,125.090263,124.319228,123.783613,123.327441,123.233338,150.816170,173.763737,143.258121,154.764279,...,134.191961,138.613617,130.747753,133.736875,136.727837,142.664401,140.440584,135.848213,166.616136,158.257726
2025-02-14,130.089869,125.156213,124.364857,123.837069,123.377747,123.270993,150.274802,173.199508,142.767944,153.891265,...,133.780589,137.823632,129.984447,133.392621,135.324742,141.341351,139.861118,135.170221,165.487620,157.645787
2025-02-17,130.261493,125.118581,124.341564,123.808394,123.352594,123.258441,149.922377,173.382813,142.738767,153.615032,...,133.924761,137.906436,129.823612,133.295083,135.394897,141.190245,139.653742,135.007735,165.651844,157.783566


In [16]:
px.line(norm_df)

In [17]:
final_NAV = norm_df.copy().iloc[-1]
final_NAV.sort_values(ascending=False, inplace=True)
final_NAV

NAV_multi_asset_ICICI_Prudential_Multi_Asset_Fund                173.930411
NAV_aggressive_hybrid_ICICI_Prudential_Equity_and_Debt_Fund      166.456123
NAV_multi_asset_UTI_Multi_Asset_Allocation_Fund                  161.401834
portfolio                                                        158.271466
NAV_multi_asset_Nippon_India_Multi_Asset_Allocation_Fund         154.982902
NAV_multi_asset_SBI_Multi_Asset_Allocation_Fund                  150.819664
NAV_multi_asset_HDFC_Multi_Asset_Fund                            143.199767
NAV_aggressive_hybrid_DSP_Aggresive_Hybrid_Fund                  142.442162
NAV_aggressive_hybrid_HDFC_Hybrid_Equity_Fund                    139.772242
NAV_equity_savings_Kotak_Equity_Savings_Fund                     138.826028
NAV_aggressive_hybrid_Canara_Robeco_Equity_Hybrid_fund           136.484141
NAV_aggressive_hybrid_SBI_Equity_Hybrid_Fund                     135.891576
NAV_conservative_hybrid_Kotak_Debt_Hybrid_Fund                   135.620883
NAV_equity_s

In [18]:
importlib.reload(utils)
utils.plot_correlation_matrix(merged_df.loc['2025':], height=1000, width=1200)

## Sharpe, Sortino and CAGR Analysis

In [20]:
for year in range(2021, 2025):
    print(f"length of year {year}: {len(merged_df.loc[f"{year}"])}")

length of year 2021: 241
length of year 2022: 243
length of year 2023: 241
length of year 2024: 240


### Let's assume an average of 242 data points (trading days) per year

In [21]:
def calculate_sharpe_sortino(prices, risk_free_rate=0.0, periods_per_year=242):
    """
    Calculate Sharpe and Sortino ratios from a pandas Series of weekday prices.
    
    Parameters:
    -----------
    prices : pd.Series
        Time series of asset prices with DatetimeIndex (weekdays only)
    risk_free_rate : float, optional
        Annualized risk-free rate, default 0
    periods_per_year : int, optional
        Number of periods in a year (242 for trading days)
        
    Returns:
    --------
    dict
        Dictionary containing Sharpe and Sortino ratios
    """
    # Ensure the index is datetime and sorted
    if not isinstance(prices.index, pd.DatetimeIndex):
        raise ValueError("Prices must have a DatetimeIndex")
    
    prices = prices.sort_index()
    
    # Calculate returns (pct_change handles irregular intervals correctly)
    returns = prices.pct_change().dropna()
    # print(f"first 5 returns: {returns[:5]}")
    # print(f"last 5 returns: {returns[-5:]}")
    
    # Annualize return and volatility
    mean_return = returns.mean() * periods_per_year
    volatility = returns.std() * np.sqrt(periods_per_year)
    
    # Calculate Sharpe ratio
    sharpe_ratio = (mean_return - risk_free_rate) / volatility
    
    # Calculate Sortino ratio (only considers downside risk)
    downside_returns = returns[returns < 0]
    downside_deviation = downside_returns.std() * np.sqrt(periods_per_year)
    
    # Handle case where there are no negative returns
    if len(downside_returns) == 0 or downside_deviation == 0:
        sortino_ratio = float('inf')  # No downside risk
    else:
        sortino_ratio = (mean_return - risk_free_rate) / downside_deviation
    
    return {
        'sharpe_ratio': sharpe_ratio,
        'sortino_ratio': sortino_ratio,
        'annualized_return': mean_return,
        'annualized_volatility': volatility,
        'downside_deviation': downside_deviation
    }

In [22]:
def calculate_cagr(price_series, freq='B'):
    """
    Calculate the Compound Annual Growth Rate (CAGR) from a pandas Series of prices.
    
    Parameters:
    -----------
    price_series : pandas.Series
        Time series of asset prices with a datetime index
    freq : str, default 'B'
        Frequency of the data. 'B' for business days (weekdays)
        Other options: 'D' for calendar days, 'M' for months, etc.
    
    Returns:
    --------
    float
        The CAGR value as a decimal (multiply by 100 for percentage)
    """
    # Ensure the series is sorted by date
    price_series = price_series.sort_index()
    
    # Get the start and end prices
    start_price = price_series.iloc[0]
    end_price = price_series.iloc[-1]
    
    # Calculate the total return
    total_return = end_price / start_price
    
    # Calculate the time period in years
    if freq == 'B':
        # For business days (approx. 242 trading days per year)
        time_years = len(price_series) / 242
    elif freq == 'D':
        # For calendar days
        days = (price_series.index[-1] - price_series.index[0]).days
        time_years = days / 365.25
    elif freq == 'M':
        # For monthly data
        months = (price_series.index[-1].year - price_series.index[0].year) * 12 + \
                 (price_series.index[-1].month - price_series.index[0].month)
        time_years = months / 12
    else:
        # Default using actual time difference
        days = (price_series.index[-1] - price_series.index[0]).days
        time_years = days / 365.25
    
    # Calculate CAGR
    cagr = (total_return ** (1 / time_years)) - 1
    
    return cagr

In [23]:
# Calculate Sharpe and Sortino
result = calculate_sharpe_sortino(nifty_df['NAV_benchmark_nifty'], risk_free_rate=0)
print(f"Sharpe Ratio: {result['sharpe_ratio']:.4f}")
print(f"Sortino Ratio: {result['sortino_ratio']:.4f}")

Sharpe Ratio: 1.1398
Sortino Ratio: 1.5192


In [24]:
# Calculate CAGR
cagr = calculate_cagr(nifty_df['NAV_benchmark_nifty'])
print(f"CAGR: {cagr:.4%}")

CAGR: 16.3873%


In [25]:
def calculate_performance_metrics(prices_df, risk_free_rate=0.0, periods_per_year=242):
    """
    Calculate performance metrics (Sharpe ratio, Sortino ratio, CAGR, Risk-Adjusted Returns) for each asset in a dataframe.
    
    Parameters:
    -----------
    prices_df : pd.DataFrame
        DataFrame containing price series for multiple assets.
        Each column represents an asset, and the index should be a DatetimeIndex with business days.
    risk_free_rate : float, optional
        Annualized risk-free rate, default 0.0
    periods_per_year : int, optional
        Number of periods in a year, default 242 (trading days)
        
    Returns:
    --------
    pd.DataFrame
        DataFrame with assets as index and performance metrics as columns
    """
    # Initialize an empty dataframe to store the results
    results = pd.DataFrame(
        index=prices_df.columns,
        columns=['Sharpe_Ratio', 'Sortino_Ratio', 'CAGR', 'Risk_Adj_Returns', 'Risk_Adj_Returns_Downside']
    )
    
    # Calculate metrics for each asset
    for asset in prices_df.columns:
        # Get the price series for this asset
        price_series = prices_df[asset]
        
        # Calculate Sharpe and Sortino ratios
        metrics = calculate_sharpe_sortino(
            price_series,
            risk_free_rate=risk_free_rate,
            periods_per_year=periods_per_year
        )
        
        # Calculate CAGR
        cagr = calculate_cagr(price_series, freq='B')
        
        # Calculate returns
        returns = price_series.pct_change().dropna()
        
        # Calculate annualized volatility (standard deviation of returns)
        annualized_volatility = returns.std() * np.sqrt(periods_per_year)
        
        # Calculate downside deviation (standard deviation of negative returns only)
        downside_returns = returns[returns < 0]
        downside_deviation = downside_returns.std() * np.sqrt(periods_per_year)
        
        # Calculate risk-adjusted returns (CAGR / volatility)
        # Adding a small constant to avoid division by zero
        risk_adjusted_returns = cagr / (annualized_volatility + 1e-10)
        
        # Calculate risk-adjusted returns based on downside deviation
        risk_adj_returns_downside = cagr / (downside_deviation + 1e-10)
        
        # Store results
        results.loc[asset, 'Sharpe_Ratio'] = np.round(metrics['sharpe_ratio'], 3)
        results.loc[asset, 'Sortino_Ratio'] = np.round(metrics['sortino_ratio'], 3)
        results.loc[asset, 'CAGR'] = np.round(cagr, 3) * 100  # Convert to percentage
        results.loc[asset, 'Risk_Adj_Returns'] = np.round(risk_adjusted_returns, 3)
        results.loc[asset, 'Risk_Adj_Returns_Downside'] = np.round(risk_adj_returns_downside, 3)
        
    return results

In [26]:
performance_metrics = calculate_performance_metrics(merged_df, risk_free_rate=0.06)
performance_metrics.sort_values('Risk_Adj_Returns_Downside', ascending=False, inplace=True)
performance_metrics

,Sharpe_Ratio,Sortino_Ratio,CAGR,Risk_Adj_Returns,Risk_Adj_Returns_Downside
NAV_arbitrage_Invesco_India_Arbitrage_Fund,0.282,0.524,6.4,7.333,13.613
NAV_arbitrage_Kotak_Equity_Arbitrage_Fund,0.243,0.437,6.4,7.29,13.121
NAV_arbitrage_HDFC_Arbitrage_Fund_Wholesale_Plan,-0.076,-0.142,6.1,6.941,12.951
NAV_arbitrage_ICICI_Prudential_Equity_Arbitrage_Fund,0.005,0.008,6.2,7.118,12.851
NAV_arbitrage_SBI_Arbitrage_Opportunities_Fund,0.04,0.072,6.2,6.873,12.412
NAV_equity_savings_ICICI_Prudential_Equity_Savings_Fund,1.226,1.567,9.9,3.459,4.422
NAV_conservative_hybrid_ICICI_Prudential_Regular_Savings_Fund,1.376,1.73,10.6,3.512,4.416
NAV_conservative_hybrid_UTI_Conservative_Hybrid_Fund,1.334,1.745,11.7,3.026,3.959
NAV_equity_savings_DSP_Equity_Savings_Fund,1.348,1.852,12.6,2.849,3.912
NAV_conservative_hybrid_SBI_Conservative_Hybrid_Fund,1.433,1.693,11.8,3.242,3.832


## We'll first find out the best fund in each category

Let's divide the whole time frame into 4 periods and then we'll calculate the metrics on each period. We'll then also look at average metrics and then decide the best fund in each category.

In [28]:
df_windows = {}
indexes = merged_df.index
for i in range(4):
    curr_indexes = indexes[i * len(indexes) // 4: (i + 1) * len(indexes) // 4]
    df_windows[i] = merged_df.loc[curr_indexes]

In [29]:
performance_metrics_dict = {}
for big_key in dfs.keys():
    performance_metrics_dict[big_key] = pd.DataFrame()
    for i in range(4):
        # print(f"Window {i}")
        performance_metrics = calculate_performance_metrics(df_windows[i][df_windows[i].columns[df_windows[i].columns.str.contains(f"{big_key}")]], risk_free_rate=0.06)
        performance_metrics.index = performance_metrics.index + f"_{i}"
        performance_metrics_dict[big_key] = pd.concat([performance_metrics_dict[big_key], performance_metrics])

In [34]:
performance_metrics_dict = {}
overall_performance_metrics_dict = {}
for big_key in dfs.keys():
    performance_metrics_dict[big_key] = {}
    overall_performance_metrics_dict[big_key] = pd.DataFrame()
    for key in dfs[big_key].keys():
        performance_metrics_dict[big_key][key] = pd.DataFrame()
        for i in range(4):
            # print(f"Window {i}")
            performance_metrics = calculate_performance_metrics(df_windows[i][df_windows[i].columns[df_windows[i].columns.str.contains(f"{big_key}_{key}")]], risk_free_rate=0.06)
            performance_metrics.index = performance_metrics.index + f"_{i}"
            performance_metrics_dict[big_key][key] = pd.concat([performance_metrics_dict[big_key][key], performance_metrics])

        overall_performance_metrics = calculate_performance_metrics(merged_df[merged_df.columns[merged_df.columns.str.contains(f"{big_key}_{key}")]], risk_free_rate=0.06)
        overall_performance_metrics_dict[big_key] = pd.concat([overall_performance_metrics_dict[big_key], overall_performance_metrics])
        overall_performance_metrics_dict[big_key].sort_values('Risk_Adj_Returns_Downside', ascending=False, inplace=True)

In [35]:
for key, df in overall_performance_metrics_dict.items():
    print(f"{key}:")
    display(df)

benchmark:


,Sharpe_Ratio,Sortino_Ratio,CAGR,Risk_Adj_Returns,Risk_Adj_Returns_Downside
NAV_benchmark_nifty_benchmark_nifty_50,0.737,0.987,16.9,1.167,1.563


arbitrage:


,Sharpe_Ratio,Sortino_Ratio,CAGR,Risk_Adj_Returns,Risk_Adj_Returns_Downside
NAV_arbitrage_Invesco_India_Arbitrage_Fund,0.282,0.524,6.4,7.333,13.613
NAV_arbitrage_Kotak_Equity_Arbitrage_Fund,0.243,0.437,6.4,7.29,13.121
NAV_arbitrage_HDFC_Arbitrage_Fund_Wholesale_Plan,-0.076,-0.142,6.1,6.941,12.951
NAV_arbitrage_ICICI_Prudential_Equity_Arbitrage_Fund,0.005,0.008,6.2,7.118,12.851
NAV_arbitrage_SBI_Arbitrage_Opportunities_Fund,0.04,0.072,6.2,6.873,12.412


multi_asset:


...

,Sharpe_Ratio,Sortino_Ratio,CAGR,Risk_Adj_Returns,Risk_Adj_Returns_Downside
NAV_conservative_hybrid_ICICI_Prudential_Regular_Savings_Fund,1.376,1.73,10.6,3.512,4.416
NAV_conservative_hybrid_UTI_Conservative_Hybrid_Fund,1.334,1.745,11.7,3.026,3.959
NAV_conservative_hybrid_SBI_Conservative_Hybrid_Fund,1.433,1.693,11.8,3.242,3.832
NAV_conservative_hybrid_HDFC_Hybrid_Debt_Fund,1.368,1.617,11.9,3.04,3.594
NAV_conservative_hybrid_Kotak_Debt_Hybrid_Fund,1.404,1.663,12.7,2.94,3.483


equity_savings:


,Sharpe_Ratio,Sortino_Ratio,CAGR,Risk_Adj_Returns,Risk_Adj_Returns_Downside
NAV_equity_savings_ICICI_Prudential_Equity_Savings_Fund,1.226,1.567,9.9,3.459,4.422
NAV_equity_savings_DSP_Equity_Savings_Fund,1.348,1.852,12.6,2.849,3.912
NAV_equity_savings_Kotak_Equity_Savings_Fund,1.229,1.496,12.5,2.599,3.165
NAV_equity_savings_HDFC_Equity_Savings_Fund,1.191,1.467,13.3,2.387,2.94
NAV_equity_savings_SBI_Equity_Savings_Fund,0.987,1.189,12.0,2.144,2.584


aggressive_hybrid:


,Sharpe_Ratio,Sortino_Ratio,CAGR,Risk_Adj_Returns,Risk_Adj_Returns_Downside
NAV_aggressive_hybrid_ICICI_Prudential_Equity_and_Debt_Fund,1.519,1.903,25.6,2.224,2.787
NAV_aggressive_hybrid_DSP_Aggresive_Hybrid_Fund,1.085,1.348,18.4,1.739,2.161
NAV_aggressive_hybrid_SBI_Equity_Hybrid_Fund,0.984,1.263,16.5,1.659,2.13
NAV_aggressive_hybrid_HDFC_Hybrid_Equity_Fund,1.039,1.317,18.1,1.672,2.118
NAV_aggressive_hybrid_Canara_Robeco_Equity_Hybrid_fund,0.943,1.168,16.7,1.568,1.941


In [ ]:
# Uncomment below code to view the performance on different periods
# big_key = 'arbitrage'
# for key, df in performance_metrics_dict[big_key].items():
#     print(f"{key}:")
#     display(df)